### Parker Dunn

## EK 381 - HW 10 - Optional Competition

### DUE: December 3rd, 2021

### PLAN

1. Generate a simple algorithm on my own - use reduced data from **PCA** along with the concept of **nearest neighbor**
2. (b) (*maybe*) try to use a series of classifiers with weighted significance
    
    a. Not sure how I would weight them  
    b. __use multiple PCA & nearest neighbor levels__!!  
        (1) pull out the misclassified data after each attempt at classification
        (2) try to classify the missed data - weight this stage more heaviliy
        (3) repeat  
    c. (*probably*) maybe use *perceptron* or *linear discriminant analysis* instead of nearest neighbors to start  
    d. ALSO, **figure out how to determine what percentage of variance is explained by each principle component**
    
    
3. Try to use a deep learning algorithm  
    a. Gotta figure out if there is an algorithm that works well with data format that I have available
    

## List of function that will be helpful:

* Error_rate
* split_data
* PCA function


* (NEW) function to calculate "distance" between images
* `average_pet()` ... probably
* 

# PART I - SETUP

In [35]:
# Import Necessary Modules

import glob
import matplotlib.pyplot as plt 
from skimage import io
import numpy as np
%matplotlib inline

# MY IMPORTS
import math
import json
from datetime import datetime
from datetime import date

### `read_data()` - copied from assignment to get the data into this script

In [3]:
#This function reads in all n images in catsfolder/ and dogsfolder/. 
#Each 64 x 64 image is reshaped into a length-4096 row vector. 
#These row vectors are stacked on top of one another to get a data matrix
#X of size n x 4096. We also generate a -1 label if the row vector corresponds
#to a cat image and a +1 label if the row vector corresponds to a dog image
#and stack these on top of one another to get a label vector y of length n.

def read_data():
    
    # get image filenames
    cat_locs = glob.glob('petdataset/catsfolder/*.jpg')
    dog_locs = glob.glob('petdataset/dogsfolder/*.jpg')
    num_cats = len(cat_locs)
    num_dogs = len(dog_locs)
    
    # initialize empty arrays
    X_cats = np.zeros((num_cats,64*64))
    X_dogs = np.zeros((num_dogs,64*64))
    y_cats = np.zeros((num_cats,1))
    y_dogs = np.zeros((num_dogs,1))
              
    #Load data, reshape into a 1D vector and set labels
    
    keep_track = 0

    for i in range(num_cats):
        img = cat_locs[i]
        im = io.imread(img)
        im = im.reshape(64*64)
        X_cats[i,:] = im
        y_cats[i] = -1.0
        keep_track += 1

    for i in range(num_dogs):
        img = dog_locs[i]
        im = io.imread(img)
        im = im.reshape(64*64)
        X_dogs[i,:] = im
        y_dogs[i] = 1.0
        keep_track += 1
    
    # combine both datasets
    X = np.append(X_cats,X_dogs,0)
    y = np.append(y_cats,y_dogs)
    
    return X, y 

### `split_data()` - copied from assignment to create testing data

In [4]:
#This function takes in a data matrix X, label vector y, and 
#the desired percentage testpercent. It randomly assigns  
#testpercent of the rows of X and y to Xtest and ytest. 
#The remaining data is assigned to Xtrain and ytrain.

def split_data(X,y,testpercent):
        
    [n, d] = X.shape
    
    ntest = int(round(n*(float(testpercent)/100)))
    ntrain = int(round(n - ntest))
        
    Xtrain = np.zeros((ntrain,d))
    Xtest = np.zeros((ntest,d))
    ytrain = np.zeros((ntrain,1))
    ytest = np.zeros((ntest,1))   
        
    Data = np.column_stack((X,y))
    Data = np.random.permutation(Data)
    
    for i in range(ntest):
        Xtest[i,:] = Data[i,0:d]
        ytest[i] = Data[i,d]
        
    for i in range(ntrain):
        Xtrain[i,:] = Data[i+ntest,0:d]
        ytrain[i] = Data[i+ntest,d]
        
    return Xtrain, ytrain, Xtest, ytest

#### Using copied functions to prepare the data

In [5]:
# Read data from the image files
X, y = read_data()

# Splitting data into data for training and testing
Xtrain, ytrain, Xtest, ytest = split_data(X,y,10)
# 10% of the data reserved for testing

### `error_rate()` - copied from assignment to eventually use for testing the model

In [6]:
#This function takes in a vector of true labels ytrue
#and a vector of guessed labels yguess and reports back
#the error rate of the guesses as a percentage 0% to 100%.

def error_rate(ytrue, yguess):
    # compare your predictions with the correct labels to determine how many of your predictions were correct.
    total = len(ytrue)
    wrong = np.sum(np.not_equal(ytrue,yguess))
    error = 100*wrong/total
    # divide the number of correct predictions by the number of total samples to determine your classification accuracy.
    return error

In [7]:
#This code implements the PCA exactly as in MATLAB so as to be consistent.
#It takes in an n x d data matrix X and returns a d x d orthonormal matrix pcaX. 
#Each column of pcaX contains a basis vector, sorted by decreasing variance.

def pca(X):
    covX = np.cov(X,rowvar=False)
    [Lambda,Vtranspose] = np.linalg.eig(covX)
    neworder = np.argsort(-abs(Lambda))
    pcaX = Vtranspose[:,neworder]
    pcaX = pcaX.real

    return pcaX

In [9]:
#This function takes in a training data matrix Xtrain, training
#label vector ytrain and uses them to the decision rule corresponding
#to a very simple one-layer neural network: the perceptron. 
#It also takes in a data matrix Xrun and produces a vector of label
#guesses yguess, corresponding to the sign of the linear prediction.

def perceptron(Xtrain,ytrain,Xrun):
    # Doing one-time calculation
    inv_Xtrain = np.linalg.pinv(Xtrain.T @ Xtrain)  # dimensions: (4096 x rows) * (rows * 4096) to work with the other vector ... it hasto be done this way
    xT_by_ytrain = Xtrain.T @ ytrain          # dimensions: (rows x 4096) * (rows * 1) -- > (4096 x rows) * (rows * 1) !!! only way this works
    multiply_vec = inv_Xtrain @ xT_by_ytrain
    
    # creating yguess
    yguess = np.zeros((Xrun.shape[0], 1))
    
    # Extracting and manipulating each row of Xrun
    for i in range(Xrun.shape[0]):
        X_vec = Xrun[i,:]
        X_arr = np.array([X_vec])
#         if (i == 0):
#             print(X_arr.shape)  # need this to be a column vector!
#             print(multiply_vec.shape)
        if ((X_arr @ multiply_vec) >= 0):
            yguess[i,0] = 1
        else:
            yguess[i,0] = -1
            
    return yguess

# Next Up: "THE FUNCTION"
### This next code block contains the `pet_classifier()` function which will be trained using another function within this script

Trying to understand the design of a classifier algorithm better...

#### What classifier needs to be able to do!
* center X
* Matrix multiplication of Xrun * Vk
* Run the data through perceptron
    * (!!) needs to have been trained OR needs Xtrain, ytrain
* Process yguess (if it has multiple columns that need to be weighted and added)
* Return yguess


#### DEPLOYMENT QUESTIONS
* What am I going to save for a trained model?
* How am I going to "re-train" the model when somone else opens the program?


#### TRAINING QUESTIONS
* Setup to test a combination of weights every time. Do I need to use multiple weight predictions for the same data?


* Do I want to use the weighting while training? (i.e. as I extract data, I can weight the data being used differently...


In [33]:
def pet_classifier(Xtrain, ytrain, Xrun, k, weights, mu):
    # Repeated steps: PCA/reduce_data() -> use classifier (e.g. k nearest neighbors) ->
    #                -> determine which guesses are wrong -> extract incorrect guess rows from X 
    #                -> *REPEAT*
    
    ytest_guesses = np.zeros((Xrun.shape[0], len(weights)))
    ytraining_guesses = np.zeros((Xtrain.shape[0], len(weights)))
    # number of rows    ==  the number of measurements
    # number of columns ==  the number of weights used
    
    # REDUCING THE DATA - PCA & data reduction
    Xtrain_cent = Xtrain - mu        # center training data
    Xrun_cent = Xrun - mu            # center testing data    
    pcaXtrain = pca(Xtrain_cent)            # Run PCA
    
    for i in range(len(weights)):
        
        wt_k = round(k * weights[i])
        print("wt_k is {}".format(wt_k))
        Vk = pcaXtrain[0:wt_k,:].T       # Extract a portion of the eigenvectors

        Xtrain_red = Xtrain_cent @ Vk
        Xrun_red = Xrun_cent @ Vk

    # CLASSIFY REDUCED DATA WITH PERCEPTRON
        yguesstrain = perceptron(Xtrain_red, ytrain, Xtrain_red)    
        yguesstest = perceptron(Xtrain_red, ytrain, Xrun_red)

    # SAVE EACH OF THE RETURNED GUESS VECTORS
        ytest_guesses[:,i] = yguesstest[:,0]
        ytraining_guesses[:,i] = yguesstrain[:,0]
    
    # "COMBINE" COLUMNS OF "ytest_guesses" AND "ytraining_guesses"
    ytest_result = np.zeros((ytest_guesses.shape[0], 1))
    ytrain_result = np.zeros((ytraining_guesses.shape[0], 1))
    
    for row in range(ytraining_guesses.shape[0]):      # "for each row"
        ytrain_result[row,0] = np.sign(np.sum(ytraining_guesses[row,:]))
        
    for row in range(ytest_guesses.shape[0]):
        ytest_result[row,0] = np.sign(np.sum(ytest_guesses[row,:]))
    
    return ytrain_result, ytest_result


In [31]:
todays_testing = {}

In [39]:
# Generate mu vector first
mu = np.mean(Xtrain, axis=0)

# PREPARING OTHER ARGUMENTS
# Xtrain comes from above
# ytrain comes from above
# Xrun is Xtest from above
k = 400
weights = [0.15, 0.3, 0.5]   # right now this represents what number of the eigenvectors will be used
# 0.1 -> 40
# 0.25 -> 100
# 0.75 -> 300

yguesstrain, yguesstest = pet_classifier(Xtrain, ytrain, Xtest, k, weights, mu)
training_error = error_rate(yguesstrain, ytrain)
test_error = error_rate(yguesstest, ytest)

# SAVING DATA FROM EACH RUN
now = datetime.now()
curtime = now.strftime("%H%M%S")
todays_testing[("ID" + curtime)] = {"weights": weights, "train_error": training_error, 
                                    "test_error": test_error}
print(todays_testing)

wt_k is 60
wt_k is 120
wt_k is 200
{'ID183330': {'weights': [0.1, 0.25, 0.75], 'train_error': 7.5, 'test_error': 7.5}, 'ID184542': {'weights': [0.15, 0.3, 0.5], 'train_error': 6.611111111111111, 'test_error': 5.5}}


In [40]:
# print(todays_testing)
# print(yguesstrain[0:20])
# print(yguesstest[0:20])

In [41]:
# SAVE ALL OF THE DAYS TESTING/TRAINING

now = datetime.now()
today = date.today()

day = today.strftime("%d%b%Y")

for key in todays_testing.keys():
    with open("TrainingData/" + day + "_" + key + ".json", 'w') as file:
        json.dump(todays_testing[key], file)

In [ ]:
"""
class Image_Classifier:
    
    def __init__(training = False):
        self.k = 400
        self.weights = [0.2, 0.35, 0.45]
        self.training = training
        calc_mu()
        get_data()
        if (not training):
            getXtrain()
            getytrain()
    
    def calc_mu(self):
        if (self.training):
            self.mu = np.mean(self.X, axis = 0)
        else:
            print("calc_mu: Not prepared yet")
    
    def getXtrain(self, xtrain):
        if (self.training):
            self.Xtrain = xtrain
        else:
            print("calc_Xtrain: Not prepared yet")
    
    def getytrain(self, ytrain):
        if (self.training):
            self.ytrain = ytrain
        else:
            print("calc_ytrain: Not prepared yet")
            
#     def define_weights(self, weights):
#         if (self.training):
#             print("define_weights: will be loading weights from somewhere")
#         else:
#             self.weights = weights
    
    def get_data(self, X):
        if (not self.training):
            self.X = X
            
    def perceptron(self)
        if (self.training):
            # use self.Xtrain
        else:
            # use self.X
    
    def pet_classifier(self, training = False):
        # Repeated steps: PCA/reduce_data() -> use classifier (e.g. k nearest neighbors) ->
        #                -> determine which guesses are wrong -> extract incorrect guess rows from X -> *REPEAT*

        # PCA & data reduction
        X_cent = self.X - self.mu        # center data
        pcaXrun = pca(X_cent)            # Run PCA

        yguess = np.zeros((self.X.shape[0], len(self.weights)))  
        # number of rows    ==  the number of measurements
        # number of columns ==  the number of weights used
        
        for wt in self.weights:
            weighted_k = wt * k
            Vk = pcaXrun[0:weighted_k,:].T     # Extract extract k features/rows and transpose 
                                               # to column eigenvectors
            X_reduced = X_cent @ Vk

            # NOW, use reduced data to make predictions
            invX

        return 0

# def pet_classifier(Xrun, mu, weights = [.1, 0.25, 0.5], training=False):
"""